# Libraries

In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
algo_name = 'MF'
data_strategies = ['uniformly_random',
                   'popularity_good',
                   'popularity_bad',
                   'popularity_good_for_bp_ur',
                   'popularity_bad_for_bp_ur']

# Analysis

## LKPY

In [ ]:
algo_versions = {"UserKNN":[{'center':True, 'min_sim':0},
                            {'center':False, 'min_sim':0},
                            {'center':True, 'min_sim':-1}],
                "MF":[{"bias":True},
                       {"bias":False}],
                "FunkSVD":[{"bias":True},
                       {"bias":False}]}

In [ ]:
versions = algo_versions[algo_name]

In [ ]:
file_location = 'experimental_results/'+algo_name+'/'

In [ ]:
results = []
for data_strategy in data_strategies:
    for args in versions:
        file = open(file_location+data_strategy+'_'+str(args)+'.pkl','rb')
        result = pkl.load(file)
        results.append(result)

In [ ]:
stringed_versions = [str(args) for args in versions]

In [ ]:
# Initialize empty lists for the two halves
bias = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(': ')
        
    bias.append(parts[-1].split('}')[0])

In [ ]:
library = ["Lenskit"]

In [ ]:
index = pd.MultiIndex.from_product([data_strategies, bias, library], names=['DataStrategy', 'Bias','Library']).drop_duplicates()

In [ ]:
results_lkpy = pd.DataFrame(results, index=index)

## Cornac

In [ ]:
algo_versions = {"CornacMF":[{"bias":True},
                       {"bias":False}]}
algo_name = 'CornacMF'

In [ ]:
versions = algo_versions[algo_name]

In [ ]:
file_location = 'experimental_results/'+algo_name+'/'

In [ ]:
results = []
for data_strategy in data_strategies:
    for args in versions:
        file = open(file_location+data_strategy+'_'+str(args)+'.pkl','rb')
        result = pkl.load(file)
        results.append(result)

In [ ]:
stringed_versions = [str(args) for args in versions]

In [ ]:
# Initialize empty lists for the two halves
bias = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(': ')
        
    bias.append(parts[-1].split('}')[0])

In [ ]:
library = ["Cornac"]

In [ ]:
index = pd.MultiIndex.from_product([data_strategies, bias, library], names=['DataStrategy', 'Bias','Library']).drop_duplicates()

In [ ]:
results_cornac = pd.DataFrame(results, index=index)

# Merge

In [ ]:
mf_metrics = pd.concat([results_lkpy, results_cornac]).reset_index().sort_values(['DataStrategy','Library','Bias']).set_index(['DataStrategy','Library','Bias']).reindex(data_strategies, level=0)

In [ ]:
metrics_order = ['pop_corr','ARP','ave_PL','ACLT','AggDiv','RMSE','NDCG']
mf_metrics = mf_metrics[metrics_order]

In [ ]:
mf_metrics = mf_metrics.rename(columns = {'pop_corr':'PopCorr','ave_PL':'PL','ACLT':'APLT','NDCG':'NDCG@10'})

In [ ]:
mf_metrics = mf_metrics.rename(index={'uniformly_random':'Scenario 1',
                         'popularity_good':'Scenario 2',
                         'popularity_bad':'Scenario 3',
                         'popularity_good_for_bp_ur':'Scenario 4',
                         'popularity_bad_for_bp_ur':'Scenario 5'})

In [ ]:
print(mf_metrics.round(3).to_latex())

In [ ]:
import pickle
with open('metrics_combined/all_mf.pkl', 'wb') as f:
    pickle.dump(mf_metrics.round(3), f)